In [17]:
pip install category_encoders

###Data Importing

In [18]:
#importing required libraries and funcitons
from pandas import read_csv,DataFrame, Series
from category_encoders import TargetEncoder
import plotly.figure_factory as ff
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import GridSearchCV
from sklearn import linear_model
from sklearn.linear_model import SGDRegressor as SGD
import joblib
from sklearn.svm import SVR
from sklearn.ensemble import RandomForestRegressor as RFR

data1 = read_csv('/content/Fossil Age train_data.csv') #importing the data
data1.info() #checking information about data

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4398 entries, 0 to 4397
Data columns (total 13 columns):
 #   Column                      Non-Null Count  Dtype  
---  ------                      --------------  -----  
 0   uranium_lead_ratio          4398 non-null   float64
 1   carbon_14_ratio             4398 non-null   float64
 2   radioactive_decay_series    4398 non-null   float64
 3   stratigraphic_layer_depth   4398 non-null   float64
 4   geological_period           4398 non-null   object 
 5   paleomagnetic_data          4398 non-null   object 
 6   inclusion_of_other_fossils  4398 non-null   bool   
 7   isotopic_composition        4398 non-null   float64
 8   surrounding_rock_type       4398 non-null   object 
 9   stratigraphic_position      4398 non-null   object 
 10  fossil_size                 4398 non-null   float64
 11  fossil_weight               4398 non-null   float64
 12  age                         4398 non-null   int64  
dtypes: bool(1), float64(7), int64(1),

###Data Encoding

In [19]:
#checking categorical values
print(data1['geological_period'].unique())
print(data1['paleomagnetic_data'].unique())
print(data1['surrounding_rock_type'].unique())
print(data1['stratigraphic_position'].unique())
print(data1['inclusion_of_other_fossils'].unique())

['Cretaceous' 'Cambrian' 'Permian' 'Devonian' 'Jurassic' 'Neogene'
 'Triassic' 'Paleogene' 'Ordovician' 'Carboniferous' 'Silurian']
['Normal polarity' 'Reversed polarity']
['Conglomerate' 'Limestone' 'Shale' 'Sandstone']
['Middle' 'Top' 'Bottom']
[False  True]


In [20]:
#data encoding using map function
data1['paleomagnetic_data']= data1['paleomagnetic_data'].map({'Normal polarity':0,'Reversed polarity':1 })
data1['stratigraphic_position']= data1['stratigraphic_position'].map({'Bottom':0,'Middle':1,'Top':2 })
data1['geological_period']= data1['geological_period'].map({'Cretaceous':8,'Cambrian':0,'Permian':5, 'Devonian':3, 'Jurassic':7, 'Neogene':10, 'Triassic':6, 'Paleogene':9, 'Ordovician':1, 'Carboniferous':4, 'Silurian':2  })
data1['inclusion_of_other_fossils'] = data1['inclusion_of_other_fossils'].map({True:1,False:0})

data1.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4398 entries, 0 to 4397
Data columns (total 13 columns):
 #   Column                      Non-Null Count  Dtype  
---  ------                      --------------  -----  
 0   uranium_lead_ratio          4398 non-null   float64
 1   carbon_14_ratio             4398 non-null   float64
 2   radioactive_decay_series    4398 non-null   float64
 3   stratigraphic_layer_depth   4398 non-null   float64
 4   geological_period           4398 non-null   int64  
 5   paleomagnetic_data          4398 non-null   int64  
 6   inclusion_of_other_fossils  4398 non-null   int64  
 7   isotopic_composition        4398 non-null   float64
 8   surrounding_rock_type       4398 non-null   object 
 9   stratigraphic_position      4398 non-null   int64  
 10  fossil_size                 4398 non-null   float64
 11  fossil_weight               4398 non-null   float64
 12  age                         4398 non-null   int64  
dtypes: float64(7), int64(5), object(1

###Feature Selection (Input and Output coulumns)

In [21]:
X = data1.drop(['age'], axis = 1) # selecting input columns
Y = data1['age'] # selecting output colomn (age)
print(X.shape)
print(Y.shape)

(4398, 12)
(4398,)


###Target Encoding

In [22]:
#using target encoding for values which cannot be ranked
Target_Encoder1 = TargetEncoder(cols=['surrounding_rock_type']).fit(X,Y)
X= Target_Encoder1.transform(X)
print(X.info())
X.head(50)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4398 entries, 0 to 4397
Data columns (total 12 columns):
 #   Column                      Non-Null Count  Dtype  
---  ------                      --------------  -----  
 0   uranium_lead_ratio          4398 non-null   float64
 1   carbon_14_ratio             4398 non-null   float64
 2   radioactive_decay_series    4398 non-null   float64
 3   stratigraphic_layer_depth   4398 non-null   float64
 4   geological_period           4398 non-null   int64  
 5   paleomagnetic_data          4398 non-null   int64  
 6   inclusion_of_other_fossils  4398 non-null   int64  
 7   isotopic_composition        4398 non-null   float64
 8   surrounding_rock_type       4398 non-null   float64
 9   stratigraphic_position      4398 non-null   int64  
 10  fossil_size                 4398 non-null   float64
 11  fossil_weight               4398 non-null   float64
dtypes: float64(8), int64(4)
memory usage: 412.4 KB
None


,uranium_lead_ratio,carbon_14_ratio,radioactive_decay_series,stratigraphic_layer_depth,geological_period,paleomagnetic_data,inclusion_of_other_fossils,isotopic_composition,surrounding_rock_type,stratigraphic_position,fossil_size,fossil_weight
0,0.738061,0.487707,0.907884,91.17,8,0,0,0.915951,39798.967851,1,50.65,432.00
1,0.560096,0.341738,1.121302,165.44,0,0,0,0.803968,40657.672384,2,48.85,353.29
2,0.424773,0.218493,0.103855,218.98,0,0,1,0.792441,40476.477273,0,37.66,371.33
3,0.349958,0.704649,0.383617,51.09,5,0,1,0.074636,40657.672384,0,39.10,232.84
4,0.886811,0.777494,0.593254,313.72,3,0,1,1.646640,40476.477273,2,90.84,277.67
5,1.079099,0.589527,0.573466,164.37,7,0,1,1.391573,40657.672384,0,51.89,312.85
6,0.467579,0.657890,0.205170,197.56,0,0,1,0.514012,40657.672384,0,43.12,426.17
7,0.344964,0.872661,0.688772,136.40,10,0,1,0.003672,39798.967851,0,96.82,454.15
8,0.407311,0.793969,0.184602,103.81,3,0,1,0.690427,39798.967851,2,44.56,208.19
9,0.302639,0.028225,0.338870,215.61,0,0,1,0.016493,40926.512358,1,98.64,157.63


###Heatmap (for finding correlation among input features)

In [23]:
corr = X.corr()

figure = ff.create_annotated_heatmap( corr.values,list(corr.columns),list(corr.columns),corr.round(2).values,showscale=False)
figure.show()

###Data Scaling

In [24]:
X_= StandardScaler().fit_transform(X) #Data Scaling,Normalizing, Standardizing

###Linear Regression Models

In [25]:
LinearRegression1 = SGD(random_state = 1, penalty = None) # Building the model
Hparameter1 = {'eta0': [.0001, .001, .01, .1, 1], 'max_iter':[10000, 20000, 30000, 40000]} #Giving list of hyperparameters
grid_search1 = GridSearchCV(estimator=LinearRegression1, param_grid=Hparameter1, scoring='r2', cv=5) # building gridsearchcv
grid_search1.fit(X_,Y) #training, testing, evaluating, ranking

best_parameters = grid_search1.best_params_
print("Best parameters: ", best_parameters)
best_result = grid_search1.best_score_
print("Best result: ", best_result)
best_model = grid_search1.best_estimator_
print("Intercept β0: ", best_model.intercept_)
print(DataFrame(zip(X.columns, best_model.coef_), columns=['Features','Coefficients']).sort_values(by=['Coefficients'],ascending=False))

Best parameters:  {'eta0': 0.01, 'max_iter': 10000}
Best result:  0.9772504972232152
Intercept β0:  [40585.80482036]
                      Features  Coefficients
0           uranium_lead_ratio  13403.781531
3    stratigraphic_layer_depth   5255.885464
1              carbon_14_ratio   1863.577812
2     radioactive_decay_series   1348.005582
5           paleomagnetic_data    797.703633
8        surrounding_rock_type     85.082469
7         isotopic_composition     58.032049
11               fossil_weight     25.404125
6   inclusion_of_other_fossils    -21.292887
10                 fossil_size    -42.903541
4            geological_period  -1370.678074
9       stratigraphic_position  -3207.545046


In [26]:
LinearRegression2 = SGD(random_state = 1, penalty = 'l1') # Building the model
Hparameter2 = {'eta0': [.0001, .001, .01, .1, 1], 'max_iter':[10000, 20000, 30000, 40000],'alpha': [.001, .01, .1, 1,10, 100]} #Giving list of hyperparameters

grid_search2 = GridSearchCV(estimator=LinearRegression2, param_grid=Hparameter2, scoring='r2', cv=5)
grid_search2.fit(X_, Y)#training, testing, evaluating, ranking

best_parameters = grid_search2.best_params_
print("Best parameters: ", best_parameters) #printing best parameters from the list
best_result = grid_search2.best_score_
print("Best result: ", best_result) #printing best r2 value
best_model = grid_search2.best_estimator_
print("Intercept β0: ", best_model.intercept_) #printing y-intercept
print(DataFrame(zip(X.columns, best_model.coef_), columns=['Features','Coefficients']).sort_values(by=['Coefficients'],ascending=False))

Best parameters:  {'alpha': 0.01, 'eta0': 0.01, 'max_iter': 10000}
Best result:  0.9772505236319622
Intercept β0:  [40585.80350388]
                      Features  Coefficients
0           uranium_lead_ratio  13403.772524
3    stratigraphic_layer_depth   5255.874050
1              carbon_14_ratio   1863.554178
2     radioactive_decay_series   1348.001185
5           paleomagnetic_data    797.686873
8        surrounding_rock_type     85.077659
7         isotopic_composition     57.900361
11               fossil_weight     25.173833
6   inclusion_of_other_fossils    -21.280726
10                 fossil_size    -42.892445
4            geological_period  -1370.664971
9       stratigraphic_position  -3207.534011


###Creating the .pkl file based on L1 model

In [ ]:
joblib.dump(best_model, "l1fossilmodel.pkl")

['l1fossilmodel.pkl']

In [27]:
LinearRegression2 = SGD(random_state = 1, penalty = 'l2') # model building
Hparameter2 = {'eta0': [.0001, .001, .01, .1, 1], 'max_iter':[10000, 20000, 30000, 40000],'alpha': [.001, .01, .1, 1,10, 100]}

grid_search2 = GridSearchCV(estimator=LinearRegression2, param_grid=Hparameter2, scoring='r2', cv=5)
grid_search2.fit(X_, Y)

best_parameters = grid_search2.best_params_
print("Best parameters: ", best_parameters)
best_result = grid_search2.best_score_
print("Best result: ", best_result)
best_model = grid_search2.best_estimator_
print("Intercept β0: ", best_model.intercept_)
#getting the list of coefficients for interpretation
print(DataFrame(zip(X.columns, best_model.coef_), columns=['Features','Coefficients']).sort_values(by=['Coefficients'],ascending=False))

#modified r2
r, c=X_.shape
print(X_.shape)
print(r,c)
print(4/5*r,c)
modified_r2 = 1-(1-best_result)*(4/5*r-1)/(4/5*r-c-1)# 4/5*r is number of rows in training set, c is number of columns
print("modified_r2: ", modified_r2)

Best parameters:  {'alpha': 0.001, 'eta0': 0.001, 'max_iter': 10000}
Best result:  0.977239456898608
Intercept β0:  [40586.34490846]
                      Features  Coefficients
0           uranium_lead_ratio  13496.604626
3    stratigraphic_layer_depth   5229.009069
1              carbon_14_ratio   1846.837685
2     radioactive_decay_series   1346.139320
5           paleomagnetic_data    799.591635
8        surrounding_rock_type    105.688705
7         isotopic_composition     30.435220
11               fossil_weight     15.040049
6   inclusion_of_other_fossils    -43.480357
10                 fossil_size    -65.199969
4            geological_period  -1383.890143
9       stratigraphic_position  -3205.672464


In [28]:
LinearRegression4 = SGD(random_state = 1, penalty = 'elasticnet')
Hparameter4 = {'eta0': [.0001, .001, .01, .1, 1], 'max_iter':[10000, 20000, 30000, 40000],'alpha': [.001, .01, .1, 1,10, 100], 'l1_ratio': [0,0.25,0.5,0.75,1]}

grid_search4 = GridSearchCV(estimator=LinearRegression4, param_grid=Hparameter4, scoring='r2', cv=5)
grid_search4.fit(X_, Y)

best_parameters = grid_search4.best_params_
print("Best parameters: ", best_parameters)
best_result = grid_search4.best_score_
print("Best result: ", best_result)
best_model = grid_search4.best_estimator_
print("Intercept β0: ", best_model.intercept_)
print(DataFrame(zip(X.columns, best_model.coef_), columns=['Features','Coefficients']).sort_values(by=['Coefficients'],ascending=False))

Best parameters:  {'alpha': 0.01, 'eta0': 0.01, 'l1_ratio': 1, 'max_iter': 10000}
Best result:  0.9772505236319622
Intercept β0:  [40585.80350388]
                      Features  Coefficients
0           uranium_lead_ratio  13403.772524
3    stratigraphic_layer_depth   5255.874050
1              carbon_14_ratio   1863.554178
2     radioactive_decay_series   1348.001185
5           paleomagnetic_data    797.686873
8        surrounding_rock_type     85.077659
7         isotopic_composition     57.900361
11               fossil_weight     25.173833
6   inclusion_of_other_fossils    -21.280726
10                 fossil_size    -42.892445
4            geological_period  -1370.664971
9       stratigraphic_position  -3207.534011


###Support Vector Regressor Model

In [ ]:
#Support Vector Regression
SVRegressor = SVR()
Hparameters = {'kernel': ['linear', 'poly', 'rbf', 'sigmoid'], 'C': [100,1000,10000]}
grid_search5 = GridSearchCV(estimator=SVRegressor, param_grid=Hparameters, scoring='r2', cv=5)
grid_search5.fit(X_, Y)

best_parameters = grid_search5.best_params_
print("Best parameters: ", best_parameters)
best_result = grid_search5.best_score_
print("Best result: ", best_result)

r, c=X_.shape
print(X_.shape)
print(r,c)
print(4/5*r,c)
modified_r2 = 1-(1-best_result)*(4/5*r-1)/(4/5*r-c-1)# 4/5*r is number of rows in training set, c is number of columns
print("modified_r2: ", modified_r2)

Best parameters:  {'C': 10000, 'kernel': 'rbf'}
Best result:  0.985392632502502


###SVR w/o L2 regularization (for report)

In [ ]:
SVRegressor = SVR()
Hparameters = {'kernel': ['linear', 'poly', 'rbf', 'sigmoid']}
grid_search6 = GridSearchCV(estimator=SVRegressor, param_grid=Hparameters, scoring='r2', cv=5)
grid_search6.fit(X_, Y)

best_parameters = grid_search6.best_params_
print("Best parameters: ", best_parameters)
best_result = grid_search6.best_score_
print("Best result: ", best_result)
best_model = grid_search6.best_estimator_
print("Intercept β0: ", best_model.intercept_)
print(DataFrame(zip(X.columns, best_model.coef_), columns=['Features','Coefficients']).sort_values(by=['Coefficients'],ascending=False))

Best parameters:  {'kernel': 'linear'}
Best result:  0.3211154593102249
Intercept β0:  [39736.54551709]
             Features                                       Coefficients
0  uranium_lead_ratio  [3151.937359221639, 454.06132803638246, 279.55...
(4398, 12)
4398 12
3518.4 12
modified_r2:  0.3187914407992769


###Random Forest Regressor (for report)

In [ ]:
RF_Regressor1 = RFR(criterion='squared_error', max_features='sqrt', random_state=1)
no_Trees = {'n_estimators': [200,250,300,350]}
grid_search7 = GridSearchCV(estimator=RF_Regressor1, param_grid=no_Trees, scoring='r2', cv=5)
grid_search7.fit(X_, Y)

best_parameters = grid_search7.best_params_
print("Best parameters: ", best_parameters)
best_result = grid_search7.best_score_
print("best_score: ", best_result)

Important_feature = Series(grid_search7.best_estimator_.feature_importances_, index=list(X)).sort_values(ascending=False) # Getting feature importances list for the best model
print(Important_feature)

Best parameters:  {'n_estimators': 250}
best_score:  0.9477644334583429
uranium_lead_ratio            0.667812
stratigraphic_layer_depth     0.120219
stratigraphic_position        0.043184
carbon_14_ratio               0.037412
radioactive_decay_series      0.031754
fossil_size                   0.022849
fossil_weight                 0.022219
isotopic_composition          0.021680
geological_period             0.017659
surrounding_rock_type         0.007187
paleomagnetic_data            0.004482
inclusion_of_other_fossils    0.003545
dtype: float64
